#### Prerequesites:
* Define schema (make figure) 
* Define structure (make figure)
* Define schema class in python (src/models/model_schemata.py)
* Convert schema + structure into model template (models/templates/p_model.pl)

#### Procedure:
* Load data
* Adjust model script for lfi accordingly
* Create evidence file (if I don't need different evidence files, move to builmodel notebook)
* LFI

Multiple instances, 1 pst, X enzymes, one sample per instance, multiple enzymes per instance, test how many enzymes can have in one instances, how computing time depends on N p::f for 60 samples

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import random

from src.models import model_schemata as schema
from src.models import build_model as build
from src.visualization import visualize as viz

In [3]:
os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
# os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

#### Training/Testing

In [17]:
# reading csv from files into dict
data = {}
data['e_ksea'] = pd.read_csv('data/processed/ebdt_data/sub_network_n/e_ksea.csv')
data['p_fc'] = pd.read_csv('data/processed/ebdt_data/sub_network_n/p_fc.csv')
data['es_interaction'] = pd.read_csv('data/processed/ebdt_data/sub_network_n/es_interaction.csv')
data['p_regulates'] = pd.read_csv('data/processed/ebdt_data/sub_network_n/p_regulates.csv')
data['e_ksea']

,enzyme,sample,tc,value,prob,p_dec,p_inc
0,ABL1,AC220,10.0,inc,0.648536,0.001000,0.648536
1,ABL1,AT13148,10.0,inc,0.590795,0.001000,0.590795
2,ABL1,AZ20,10.0,inc,0.651697,0.001000,0.651697
3,ABL1,AZD1480,10.0,inc,0.569660,0.001000,0.569660
4,ABL1,AZD3759,10.0,dec,0.836416,0.836416,0.001000
...,...,...,...,...,...,...,...
483,SRC,Torin,12.0,inc,0.774683,0.001000,0.774683
484,SRC,Trametinib,12.0,inc,0.735235,0.001000,0.735235
485,SRC,U73122,12.0,inc,0.550325,0.001000,0.550325
486,SRC,Ulixertinib,12.0,inc,0.601812,0.001000,0.601812


In [5]:
data['p_fc']['phosphosite'].value_counts()

ABI1(Y213)       61
PTK2(S910)       61
SRC(S75)         61
SRC(S17)         61
RBM39(Y95)       61
PXN(Y88)         61
PXN(Y118)        61
PTTG1IP(Y174)    61
PTPRG(S995)      61
PTK2(S722)       61
ABL1(S569)       61
PTK2(S29)        61
HIPK2(Y361)      61
BCLAF1(Y284)     61
ATF1(S198)       61
ANXA2P2(Y24)     61
ABL1(T735)       61
ABL1(S718)       61
PTK2(S843)       61
Name: phosphosite, dtype: int64

Splitting data into training/testing

In [18]:
# getting sample names (union of all samples in all datasets)
samples = list(set(data['e_ksea']['sample']).union(set(data['p_fc']['sample'])))
samples.sort()
# sample x% of samples randomly without replacement with seed
random.seed(612)
train = random.sample(samples, int(len(samples)*1))
print(train)
# filter data
training_data = {}
training_data['e_ksea'] = data['e_ksea'][data['e_ksea']['sample'].isin(train)].reset_index(drop=True)
training_data['p_fc'] = data['p_fc'][data['p_fc']['sample'].isin(train)].reset_index(drop=True)

['TAK715', 'TGX221', 'CHIR99021', 'CAL101', 'Ripasudil', 'KN93', 'LY2584702', 'LY2090314', 'Ku0063794', 'SP600125', 'Ribociclib', 'U73122', 'AZD5438', 'GDC0994', 'Edelfosine', 'PF3758309', 'Amuvatinib', 'AT13148', 'AZD3759', 'Go6976', 'JNJ', 'NU7441', 'GSK2334470', 'Ipatasertib', 'AZD5363', 'LY2835219', 'Torin', 'Bosutinib', 'H89', 'DNAPK', 'GDC0941', 'FRAX486', 'CX4945', 'HS173', 'Linsitinib', 'Dabrafenib', 'AZD1480', 'BX912', 'AZ20', 'TBCA', 'Selumetinib', 'GF109203X', 'AZD6482', 'AC220', 'PH797804', 'AZD8055', 'Trametinib', 'JNK', 'GSK690693', 'KD025', 'Vemurafenib', 'PF4708671', 'Tofacitinib', 'MK2206', 'GO6983', 'Dasatinib', 'AZD6738', 'PIK294', 'PD153035', 'KN62', 'Ulixertinib']


In [11]:
# Mapping data to Problog predicates
predicates = {}
predicates['e_ksea'] = schema.EKseaPredicate()
predicates['e_ksea'].add_data(training_data['e_ksea'], 'enzyme', 'sample', 'value')
predicates['p_fc'] = schema.PFoldChangePredicate()
predicates['p_fc'].add_data(training_data['p_fc'], 'phosphosite', 'sample', 'value')

In [12]:
samples = list(set(training_data['e_ksea']['sample']).union(set(training_data['p_fc']['sample'])))
enzymes = list(set(data['p_regulates']['protein'].tolist() + data['es_interaction']['enzyme'].tolist()))
evidence_dict = {}
for s in samples:
    evidence_dict[s] = {}
    for e0 in enzymes:
        evidence_dict[s][e0] = {}
        phosphosites = data['p_regulates']['phosphosite'][data['p_regulates']['protein'] == e0].tolist()
        for p in phosphosites:
            evid_generator_e = build.ProblogStatementGenerator(predicates['e_ksea'])
            evidence_e0 = evid_generator_e.generate_facts(build.EvidenceTemplate, select=[s, e0])
            evid_generator_p = build.ProblogStatementGenerator(predicates['p_fc'])
            evidence_p = evid_generator_p.generate_facts(build.EvidenceTemplate, select=[s, p])
            enzymes_p = data['es_interaction']['enzyme'][data['es_interaction']['phosphosite'] == p].tolist()
            evidence_e = []
            for e in enzymes_p:
                evidence_e.extend(evid_generator_e.generate_facts(build.EvidenceTemplate, select=[s, e]))
            evidence_dict[s][e0][p] = evidence_e0 + evidence_p + evidence_e
        if len(phosphosites) == 0:
            evid_generator_e = build.ProblogStatementGenerator(predicates['e_ksea'])
            evidence_e0 = evid_generator_e.generate_facts(build.EvidenceTemplate, select=[s, e0])
            evidence_dict[s][e0][e0] = evidence_e0

In [9]:
# write evidence_dict to text file: loop over samples and separate by line of hyphens
with open('models/ebdt_data/sub_network/n_model/evidence_exp2.pl', 'w') as f:
    for s in evidence_dict:
        for e in evidence_dict[s]:
            for p in evidence_dict[s][e]:
                for fact in evidence_dict[s][e][p]:
                    f.write(fact + '\n')
                f.write('--------------------' + '\n')

In [13]:
# write evidence_dict to text file: loop over samples and separate by line of hyphens
with open('models/ebdt_data/sub_network/n_model/evidence_exp2_training.pl', 'w') as f:
    for s in evidence_dict:
        for e in evidence_dict[s]:
            for p in evidence_dict[s][e]:
                for fact in evidence_dict[s][e][p]:
                    f.write(fact + '\n')
                f.write('--------------------' + '\n')

LFI

In [19]:
DIR = 'models/ebdt_data/sub_network/n_model/'
max_iter = 1
# learning from interpretation
cmd = f'problog lfi {DIR}n_model_lfi_exp2.pl {DIR}evidence_exp2.pl -O {DIR}n_model_exp2_1i.pl --logger {DIR}log_exp2-1i.txt -k ddnnf -v -n {max_iter}'
os.system(cmd)

DSharpError: DSharp has encountered an error. This is a known issue. See KNOWN_ISSUES for details on how to resolve this problem.


256

In [4]:
DIR = 'models/ebdt_data/sub_network/n_model/'
# learning from interpretation
cmd = f'problog ground {DIR}n_model_lfi_exp2.pl -o {DIR}n_model_exp2_ground.pl --format pl'
os.system(cmd)

(True, '0.001::occ_dec("ABI1(Y213)","AT13148").\nt(0.5,"ABL1","ABI1(Y213)")::targets("ABL1","ABI1(Y213)").\n0.001::occ_dec("ABI1(Y213)","AT13148") :- targets("ABL1","ABI1(Y213)").\n0.001::act_dec("ABL1","AT13148").\n0.001::act_dec("ABL1","AT13148").\n0.001::act_dec("ABL1","AT13148").\n0.001::act_dec("ABL1","AT13148").\n0.001::occ_dec("ABL1(S569)","AT13148").\n0.001::occ_inc("ABL1(S569)","AT13148").\nt(1.0,"ABL1(S569)")::p_occupancy("ABL1(S569)","AT13148",dec); t(0.0,"ABL1(S569)")::p_occupancy("ABL1(S569)","AT13148",inc) :- occ_dec("ABL1(S569)","AT13148"), \\+occ_inc("ABL1(S569)","AT13148").\nt(0.0,"ABL1(S569)")::p_occupancy("ABL1(S569)","AT13148",dec); t(1.0,"ABL1(S569)")::p_occupancy("ABL1(S569)","AT13148",inc) :- \\+occ_dec("ABL1(S569)","AT13148"), occ_inc("ABL1(S569)","AT13148").\nt(0.5,"ABL1(S569)")::p_occupancy("ABL1(S569)","AT13148",dec); t(0.5,"ABL1(S569)")::p_occupancy("ABL1(S569)","AT13148",inc) :- occ_dec("ABL1(S569)","AT13148"), occ_inc("ABL1(S569)","AT13148").\nt(0.0,"ABL1(

256